In [363]:
import pandas as pd
import numpy as np
import scipy
import einops
from thefuzz import process
from tqdm import tqdm
import requests
import json
import pickle

We first download the "Ausgrid average electricity consumption by LGA 2021 excel" file from https://www.ausgrid.com.au/Industry/Our-Research/Data-to-share/Average-electricity-use and save to the data folder

In [364]:
elec_con_data = pd.read_excel(
    "data/Ausgrid average electricity consumption by LGA 2021.xlsx",
    usecols="B:Q",
    skiprows=29,
    nrows=9,
    names=[
        "LGA", "Residential Daily average (kWh)", "Residential General Supply (MWh)", "Residential Off Peak Hot Water (MWh)", "Residential Total (MWh)", "Off Peak Residential Customer number", "Total Residential Customer number",
        "Number of Residential Solar Customers", "Number of Non-Residential Solar Customers", "Residential Generation Capacity (kWp)", "Non-Residential Generation Capacity (kWp)", "Energy exported to the grid (MWh)",
        "Non-Residential Small-Medium Sites MWh", "Number of Non-Residential Small-Medium Sites",
        "Non-Residential Large Sites MWh", "Number of Non-Residential Large Sites",
    ]
)
elec_con_data

,LGA,Residential Daily average (kWh),Residential General Supply (MWh),Residential Off Peak Hot Water (MWh),Residential Total (MWh),Off Peak Residential Customer number,Total Residential Customer number,Number of Residential Solar Customers,Number of Non-Residential Solar Customers,Residential Generation Capacity (kWp),Non-Residential Generation Capacity (kWp),Energy exported to the grid (MWh),Non-Residential Small-Medium Sites MWh,Number of Non-Residential Small-Medium Sites,Non-Residential Large Sites MWh,Number of Non-Residential Large Sites
0,CENTRAL COAST*,16.508265,749993.75163,175439.24757,925432.99920,80849.438356,153585.665753,27926,866,129951.235,25729.240,94069.939235,208282.67026,12157.939726,507257.00212,709.723288
1,CESSNOCK,18.009241,145062.19126,22340.42324,167402.61450,10649.386301,25466.775342,6119,269,34436.875,11315.320,32556.795154,38708.85680,1843.106849,73069.18906,135.123288
2,LAKE MACQUARIE,16.913623,433830.86633,106992.88358,540823.74991,53160.884932,87604.465753,19815,539,96355.520,12551.950,71404.042118,118244.94451,6795.572603,189172.40852,391.958904
3,MAITLAND,16.812069,186156.22587,27389.42481,213545.65068,13031.569863,34799.797260,8541,268,46886.140,8038.030,36534.991861,59161.52808,2959.734247,91106.93447,170.602740
4,MUSWELLBROOK,20.195877,43567.83267,10159.80349,53727.63616,4443.449315,7288.567123,1306,94,7703.790,1825.385,6401.202701,28306.77256,1347.082192,19973.42688,52.008219
5,NEWCASTLE,13.670967,311806.04897,53494.28002,365300.32899,29693.865753,73207.901370,11139,561,49787.295,20393.255,46486.583101,156675.75499,8035.923288,263849.58008,515.241096
6,PORT STEPHENS,16.915889,169997.82293,46769.07573,216766.89866,23513.298630,35107.934247,8151,245,40727.000,5935.905,31131.420972,53077.87271,2985.665753,89789.91025,194.471233
7,SINGLETON,21.464937,64190.69011,13978.25635,78168.94646,6087.583562,9977.271233,2447,144,15021.225,5561.590,12941.132405,27270.28062,1676.934247,32676.20314,92.002740
8,UPPER HUNTER,20.126945,34744.97519,7854.87092,42599.84611,3524.008219,5798.789041,1143,109,6751.025,1868.590,5986.931273,20682.52559,1251.331507,8937.49130,26.515068


Allocating residential usages according to five classes

In [365]:
def alloc_mean(x, allocation):
    return (1 / len(allocation)) * sum([x * a for a in allocation])

def f(x, allocation, mu):
    return sum(abs(alloc_mean(x, allocation) - mu))

N = 5
rng = np.random.default_rng(50)
allocation = rng.dirichlet([1.5 for i in range(N)])
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, elec_con_data.loc[:, "Residential Daily average (kWh)"]))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.15889974, 0.18347015, 0.36256272, 0.1651147 , 0.12995269]), mu0=array([ 82.54132513,  90.04620605,  84.56811482,  84.06034257,
       100.97938367,  68.35483605,  84.57944433, 107.32468435,
       100.63472418]): alloc_mean(mu0, allocation)=array([16.50826503, 18.00924121, 16.91362296, 16.81206851, 20.19587673,
       13.67096721, 16.91588887, 21.46493687, 20.12694484])
np.array([mu0 * a for a in allocation])=array([[13.11579506, 14.30831869, 13.43785142, 13.35716654, 16.04559776,
        10.86156564, 13.43965167, 17.05386439, 15.99083146],
       [15.14386933, 16.52079096, 15.51572474, 15.42256369, 18.52670271,
        12.54107205, 15.51780337, 19.69087597, 18.46346798],
       [29.92640729, 32.64739733, 30.66124567, 30.47714639, 36.61135994,
        24.78291523, 30.66535333, 38.9119294 , 36.48639925],
       [13.62878632, 14.8679525 , 13.9634391 , 13.87959843, 16.67318087,
        11.2863884 , 13.96530977, 17.7208833 , 16.61627252],
       [10.72646712, 11.7017465

Now we generate synthetic data based on these statistics

In [366]:
res_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    res_X[i] = rng.normal(loc=np.array([m0 * a for a in allocation]), scale=5, size=(round(elec_con_row["Total Residential Customer number"]) - round(elec_con_row["Number of Residential Solar Customers"]), N)).T

Doing the same with small-medium non-residential but this time with only 2 classes (small and medium)

In [367]:
N = 2
allocation = rng.dirichlet([1.5 for i in range(N)])
non_res_small_med_daily_avg = elec_con_data.loc[:, "Non-Residential Small-Medium Sites MWh"] / 365 * 1000 / elec_con_data.loc[:, "Number of Non-Residential Small-Medium Sites"]
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, non_res_small_med_daily_avg))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.35038753, 0.64961247]), mu0=array([ 93.87074876, 115.07923428,  95.34406327, 109.52765673,
       115.14190004, 106.83243939,  97.41132554,  89.10677057,
        90.56665378]): alloc_mean(mu0, allocation)=array([46.93537438, 57.53961714, 47.67203163, 54.76382836, 57.57095002,
       53.4162197 , 48.70566277, 44.55338529, 45.28332689])
np.array([mu0 * a for a in allocation])=array([[32.89113938, 40.32232814, 33.4073704 , 38.37712462, 40.34428544,
        37.43275409, 34.13171332, 31.22190085, 31.73342571],
       [60.97960938, 74.75690614, 61.93669286, 71.15053211, 74.7976146 ,
        69.39968531, 63.27961223, 57.88486972, 58.83322806]])


And the synthetic data for the small-medium sites

In [368]:
small_med_non_res_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    small_med_non_res_X[i] = rng.normal(
        loc=np.array([m0 * a for a in allocation]),
        scale=5,
        size=(round(elec_con_row["Number of Non-Residential Small-Medium Sites"]) - round(5 * elec_con_row["Number of Non-Residential Solar Customers"] / 6), N)
    ).T
small_med_non_res_X

{0: array([[28.94899252, 33.16449798, 32.04785134, ..., 35.97653504,
         31.8533323 , 35.76121309],
        [61.55039359, 52.99865403, 48.67683852, ..., 55.039619  ,
         63.13294479, 56.74930812]]),
 1: array([[40.79940866, 40.54317718, 39.11615061, ..., 38.25363211,
         36.27016783, 42.06026578],
        [89.74593242, 72.82016918, 74.37823425, ..., 66.47123383,
         73.20935307, 71.62920542]]),
 2: array([[30.18430944, 36.09877195, 35.27771175, ..., 42.33186365,
         24.74491567, 32.97138643],
        [60.15083652, 60.8708456 , 65.90287113, ..., 64.91252982,
         62.23840028, 59.08293296]]),
 3: array([[39.17099924, 33.39510836, 37.17608999, ..., 31.25851129,
         34.10402254, 32.3088194 ],
        [78.5302912 , 65.7103279 , 68.04183496, ..., 64.84379198,
         67.50279483, 68.61007794]]),
 4: array([[34.83762267, 48.33270675, 21.06570829, ..., 47.73570862,
         33.40962276, 37.59191925],
        [78.5971965 , 70.998388  , 68.18017665, ..., 65.651

We will assign a single mean for large non-residential sites

In [369]:
non_res_large_daily_avg = elec_con_data.loc[:, "Non-Residential Large Sites MWh"] / 365 * 1000 / elec_con_data.loc[:, "Number of Non-Residential Large Sites"]
print(f"{non_res_large_daily_avg.tolist()=}")

non_res_large_daily_avg.tolist()=[1958.1507827476653, 1481.5326248986207, 1322.2829379652603, 1463.09514164124, 1052.174412895749, 1402.985063941339, 1264.9673191795107, 973.0562859950568, 923.4853585451541]


And finally, we generate the synthetic data

In [370]:
large_non_res_X = {}
for (i, elec_con_row), avg in zip(elec_con_data.iterrows(), non_res_large_daily_avg):
    large_non_res_X[i] = rng.normal(
        loc=avg,
        scale=5,
        size=round(elec_con_row["Number of Non-Residential Large Sites"]) - round(elec_con_row["Number of Non-Residential Solar Customers"] / 6)
    ).T
large_non_res_X

{0: array([1961.60226885, 1960.75087777, 1956.09160858, 1958.77326311,
        1959.06279823, 1956.72325896, 1954.76293143, 1951.95585546,
        1955.26206227, 1957.17470632, 1963.17370686, 1946.9354752 ,
        1953.35470598, 1957.00576986, 1961.36462268, 1952.02469974,
        1955.63631826, 1952.37435847, 1955.78184136, 1957.91578909,
        1957.97903719, 1955.09545957, 1955.25662806, 1955.49654148,
        1960.32873639, 1959.37795938, 1954.42952682, 1956.64461766,
        1964.47523258, 1961.89915167, 1956.86008255, 1962.07360142,
        1957.50279396, 1955.55355959, 1958.50469389, 1963.71041087,
        1969.32901723, 1970.25102762, 1958.15452355, 1956.90967597,
        1954.03332007, 1959.003904  , 1961.61143537, 1961.37036657,
        1957.48258712, 1956.87200279, 1954.36590381, 1967.6914649 ,
        1959.5128675 , 1965.74538573, 1962.31397343, 1951.51028494,
        1956.43741788, 1957.01295911, 1948.53728536, 1956.08607855,
        1953.9781921 , 1970.11130531, 1955.23

Finding solar energy exported, by data and split across residential and non-residential

In [371]:
n_res_solar_customers = elec_con_data.loc[:, "Number of Residential Solar Customers"]
n_non_res_solar_customers = elec_con_data.loc[:, "Number of Non-Residential Solar Customers"]

In [372]:
total_solar_generation_capacity = elec_con_data.loc[:, "Residential Generation Capacity (kWp)"] + elec_con_data.loc[0, "Non-Residential Generation Capacity (kWp)"]
p_res_solar_gen_capacity = elec_con_data.loc[:, "Residential Generation Capacity (kWp)"] / total_solar_generation_capacity
p_non_res_solar_gen_capacity = elec_con_data.loc[:, "Non-Residential Generation Capacity (kWp)"] / total_solar_generation_capacity

In [373]:
avg_res_solar_gen = elec_con_data.loc[:, "Energy exported to the grid (MWh)"] * p_res_solar_gen_capacity / 365 * 1000 / n_res_solar_customers
avg_res_solar_gen

0    7.703632
1    8.343343
2    7.792037
3    7.566992
4    3.094242
5    7.538154
6    6.412714
7    5.340947
8    2.982740
dtype: float64

Generate synthetic data

In [374]:
res_solar_X = {}
for (i, elec_con_row), avg in zip(elec_con_data.iterrows(), avg_res_solar_gen):
    res_solar_X[i] = rng.normal(
        loc=-avg,
        scale=5,
        size=round(elec_con_row["Number of Residential Solar Customers"])
    ).T
res_solar_X

{0: array([ -7.32688448,  -4.01124142, -12.33710676, ...,  -7.51615368,
         -9.43121169,  -9.22362639]),
 1: array([-21.84504973, -13.5070107 , -12.98518432, ...,  -8.4764869 ,
        -12.62829163,  -5.28811189]),
 2: array([-15.58461328, -17.42533274, -16.05496493, ...,  -8.32170348,
         -4.07189762,  -6.06541661]),
 3: array([  1.07878756,   0.48557864, -10.04577144, ...,  -4.53003983,
         -3.93020353, -13.3413047 ]),
 4: array([-7.83664719, -5.84801434,  0.26975912, ...,  7.83408987,
        -4.08836736, -4.75837911]),
 5: array([ -2.62201846,  -9.53396261,  -4.91086695, ...,  -7.69569225,
        -14.9625762 , -10.0994577 ]),
 6: array([ -6.99055879, -10.27047766, -20.05814181, ..., -13.52847919,
        -12.94652589,  -1.55059948]),
 7: array([  1.53062529,  -5.27581568, -12.6675766 , ...,  -6.97113108,
         -4.11527652,  -3.99703449]),
 8: array([ 1.34462276, -8.53258515, -6.12186279, ..., -3.23851519,
        -8.81987   ,  8.35859956])}

In [375]:
avg_non_res_solar_gen = elec_con_data.loc[:, "Energy exported to the grid (MWh)"] * p_non_res_solar_gen_capacity / 365 * 1000 / n_non_res_solar_customers
avg_non_res_solar_gen

0    49.185027
1    62.360756
2    37.315642
3    41.343036
4    10.186374
5    61.307894
6    31.095044
7    33.603408
8     8.657238
dtype: float64

Split non-residential into three groups

In [376]:
N = 3
allocation = rng.dirichlet([1.5 for i in range(N)])
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, avg_non_res_solar_gen))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.32723532, 0.26016988, 0.41259481]), mu0=array([147.55508199, 187.08226883, 111.94692516, 124.02910778,
        30.55912084, 183.92368074,  93.28513105, 100.81022395,
        25.97171456]): alloc_mean(mu0, allocation)=array([49.18502733, 62.36075628, 37.31564172, 41.34303593, 10.18637361,
       61.30789358, 31.09504368, 33.60340798,  8.65723819])
np.array([mu0 * a for a in allocation])=array([[48.28523374, 61.21992518, 36.63298732, 40.58670416, 10.00002353,
        60.18632361, 30.52618925, 32.98866539,  8.4988622 ],
       [38.38938754, 48.67317088, 29.12521776, 32.26863772,  7.95056271,
        47.8514014 , 24.26998107, 26.22778357,  6.75705778],
       [60.88046071, 77.18917278, 46.18872009, 51.1737659 , 12.60853459,
        75.88595573, 38.48896073, 41.59377499, 10.71579458]])


Generate synthetic data

In [377]:
non_res_solar_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    non_res_solar_X[i] = rng.normal(
        loc=np.array([-m0 * a for a in allocation]),
        scale=5,
        size=(round(elec_con_row["Number of Non-Residential Solar Customers"]), N)
    ).T
non_res_solar_X

{0: array([[-43.28894833, -41.58860509, -50.51514241, ..., -39.00494873,
         -49.98357981, -55.1572236 ],
        [-42.85131625, -31.17905821, -33.13520226, ..., -38.98730195,
         -45.52241918, -41.18075949],
        [-63.77974125, -61.0831644 , -62.50305389, ..., -60.29953681,
         -58.45786511, -57.85431469]]),
 1: array([[-62.52986209, -60.4821967 , -64.80275702, -58.84490144,
         -65.0619954 , -52.79499528, -59.26143339, -69.94525431,
         -55.09246115, -70.75071444, -52.6418347 , -62.42220995,
         -73.06756772, -57.83785957, -60.97926677, -51.15514022,
         -59.84129194, -62.76346517, -70.88917006, -66.70035785,
         -63.15149455, -48.67549333, -57.21972863, -63.4998608 ,
         -55.95599052, -55.13133136, -60.11409812, -65.42915699,
         -61.23569221, -68.32793435, -55.29044849, -61.24198676,
         -62.39087614, -71.1842095 , -65.39716088, -61.82420864,
         -57.27800443, -53.42825513, -59.28845139, -56.14298327,
         -66.47506

And save all synthetic data into a pickle file

In [379]:
synthetic_X = {
    "residential consumption": res_X,
    "residential generation": res_solar_X,
    "non-residential small-medium site consumption": small_med_non_res_X,
    "non-residential large size consumption": large_non_res_X,
    "non-residential generation": non_res_solar_X,
}
with open("data/electricity_consumption_2020-2021.pkl", 'wb') as f:
    pickle.dump(synthetic_X, f)

In [383]:
mem_size = 0
nitems = 0
for kd, vd in synthetic_X.items():
    for k, v in vd.items():
        mem_size += v.itemsize * v.size
        nitems += v.size
print(f"memory size = {mem_size * 1e-6}")
print(f"{nitems=}")

memory size = 15.214784
nitems=1901848


In [386]:
total = elec_con_data["Total Residential Customer number"].round().sum() + elec_con_data["Number of Residential Solar Customers"].round().sum() + elec_con_data["Number of Non-Residential Small-Medium Sites"].round().sum() + elec_con_data["Number of Non-Residential Large Sites"].round().sum()
total * 365

204679955.0

We will save weather data for each of the LGAs

First we get LGA information from https://www.olg.nsw.gov.au/public/local-government-directory/ by downloading the "All NSW Council Contact Details – XLS" file and saving to the data folder. Then we match
LGAs to those in `elec_con_data` and get the postcodes

In [272]:
lga_contacts = pd.read_excel("data/LGDGPALL.xls")
lga_contacts

,ABS,ORGNAME,POSTAL_ADD1,POSTAL_ADD2,POSTAL_SUBURB,POSTAL_STATE,POSTAL_PCODE,STREET_ADD1,STREET_ADD2,STREET_SUBURB,...,MAYOR_SAL,MAYOR_FIRST,MAYOR_LAST,MAYOR_AWARD,EMAIL,WEB,AREA,POPULATION,ABN,METRO
0,10050,Albury City Council,PO Box 323,NaN,ALBURY,NSW,2640,553 Kiewa Street,NaN,Albury,...,Clr,Kylie,King,NaN,info@alburycity.nsw.gov.au,http://www.alburycity.nsw.gov.au,306,52949,92 965 474 349,0
1,10180,Armidale Regional Council,PO Box 75A,NaN,ARMIDALE,NSW,2350,135 Rusden Street,NaN,Armidale,...,Clr,Sam,Coupland,NaN,council@armidale.nsw.gov.au,http://www.armidaleregional.nsw.gov.au,8621,30594,39 642 954 203,0
2,10250,Ballina Shire Council,PO Box 450,NaN,BALLINA,NSW,2478,40 Cherry Street,NaN,Ballina,...,Clr,Sharon,Cadwallader,NaN,council@ballina.nsw.gov.au,http://www.ballina.nsw.gov.au,485,43457,53 929 887 369,0
3,10300,Balranald Shire Council,PO Box 120,NaN,BALRANALD,NSW,2715,70 Market Street,NaN,Balranald,...,NaN,NaN,NaN,NaN,council@balranald.nsw.gov.au,http://www.balranald.nsw.gov.au,21691,2341,74 678 751 581,0
4,10470,Bathurst Regional Council,Private Mail Bag 17,NaN,BATHURST,NSW,2795,158 Russell Street,NaN,Bathurst,...,Clr,Robert,Taylor,NaN,council@bathurst.nsw.gov.au,http://www.bathurst.nsw.gov.au,3818,42779,42 173 522 302,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,18350,Wingecarribee Shire Council,PO Box 141,NaN,MOSS VALE,NSW,2577,Civic Centre,68 Elizabeth Street,Moss Vale,...,NaN,NaN,NaN,NaN,mail@wsc.nsw.gov.au,http://www.wsc.nsw.gov.au,2689,49649,49 546 344 354,0
124,18400,Wollondilly Shire Council,PO Box 21,NaN,PICTON,NSW,2571,62-64 Menangle Street,NaN,Picton,...,Clr,Matthew,Gould,NaN,council@wollondilly.nsw.gov.au,http://www.wollondilly.nsw.gov.au,2555,51002,93 723 245 808,1
125,18450,Wollongong City Council,Locked Bag 8821,NaN,WOLLONGONG DC,NSW,2500,41 Burelli Street,NaN,Wollongong,...,Clr,Gordon,Bradbery,AM,council@wollongong.nsw.gov.au,http://www.wollongong.nsw.gov.au,684,213132,63 139 525 939,0
126,18500,Woollahra Municipal Council,PO Box 61,NaN,DOUBLE BAY,NSW,1360,536 New South Head Road,NaN,Double Bay,...,Clr,Susan,Wynne,NaN,records@woollahra.nsw.gov.au,http://www.woollahra.nsw.gov.au,12,58456,32 218 483 245,1


In [273]:
idx = []
for LGA in elec_con_data['LGA']:
    orgname, score, id = process.extractOne(LGA, lga_contacts['ORGNAME'])
    idx.append(id)
    print(f"{LGA=}: {orgname=}")
idx = np.array(idx)

LGA='CENTRAL COAST*': orgname='Central Coast Council'
LGA='CESSNOCK': orgname='Cessnock City Council'
LGA='LAKE MACQUARIE': orgname='Lake Macquarie City Council'
LGA='MAITLAND': orgname='Maitland City Council'
LGA='MUSWELLBROOK': orgname='Muswellbrook Shire Council'
LGA='NEWCASTLE': orgname='Newcastle City Council'
LGA='PORT STEPHENS': orgname='Port Stephens Council'
LGA='SINGLETON': orgname='Singleton Council'
LGA='UPPER HUNTER': orgname='Upper Hunter Shire Council'


In [274]:
lga_postcodes = lga_contacts.loc[idx, "POSTAL_PCODE"]

postcode_data = pd.read_csv(
    "data/AU.txt",
    sep='\t',
    names=["country code","postal code","place name","admin name1","admin code1","admin name2","admin code2","admin name3","admin code3","latitude","longitude","accuracy"]
)
postcode_data = postcode_data.drop(postcode_data[postcode_data['postal code'].duplicated()].index)  # we will use just one of the postcodes
postcode_data

,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,AU,200,Australian National University,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2777,149.1189,1.0
1,AU,221,Barton,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.3049,149.1412,4.0
2,AU,2540,Jervis Bay,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.1499,150.6969,4.0
5,AU,2600,Russell,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2991,149.1515,4.0
17,AU,2601,Canberra,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2835,149.1281,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16868,AU,6989,Maddington,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0500,115.9833,4.0
16869,AU,6990,Gosnells,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0810,116.0054,4.0
16870,AU,6991,Kelmscott,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1243,116.0259,4.0
16871,AU,6992,Armadale,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1461,116.0093,4.0


In [284]:
weather_data = {
    "structure": {"postcode": {"day": ["time","temperature_2m_max","temperature_2m_min","sunrise","sunset","precipitation_sum","precipitation_hours","shortwave_radiation_sum"]}},
    "daily units": {"time":"iso8601","temperature_2m_max":"Celcius","temperature_2m_min":"Celcius","sunrise":"iso8601","sunset":"iso8601","precipitation_sum":"mm","precipitation_hours":"h","shortwave_radiation_sum":"MJ/m^2"}
}

for postcode in (pbar := tqdm(lga_postcodes)):
    pbar.set_postfix_str(f"{postcode=}")
    postcode_row = postcode_data[postcode == postcode_data['postal code']]
    latitude, longitude = postcode_row.latitude.item(), postcode_row.longitude.item()
    r = requests.get(f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date=2021-07-01&end_date=2022-06-30&daily=temperature_2m_max,temperature_2m_min,sunrise,sunset,precipitation_sum,precipitation_hours,shortwave_radiation_sum&timezone=Australia%2FSydney")
    if r.ok:
        postcode_weather = r.json()
        weather_data[str(postcode)] = {day: other_data for day, *other_data in zip(*[v for v in postcode_weather['daily'].values()])}
    else:
        tqdm.write(f"Postcode {postcode} weather data request failed")

with open(f"data/central_coast_hunter_weather.json", 'w') as f:
    json.dump(weather_data, f)

100%|██████████| 9/9 [00:13<00:00,  1.49s/it, postcode=2337]
